In [2]:
import torch
from torchvision import datasets, transforms
import pandas as pd
#!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
#!pip install scikit-learn

import sklearn
'''from google.colab import files
src = list(files.upload().values())[0]
open('.py','wb').write(src)
import mylib'''
import io
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import math
import copy

In [3]:
data = pd.read_csv('train_data_QUICK_START.csv/train_data_QUICK_START.csv')
sequences = data['sequence']
reactivity = data.iloc[:,4:-1]
print(reactivity)

        reactivity_0001  reactivity_0002  reactivity_0003  reactivity_0004  \
0                   NaN              NaN              NaN              NaN   
1                   NaN              NaN              NaN              NaN   
2                   NaN              NaN              NaN              NaN   
3                   NaN              NaN              NaN              NaN   
4                   NaN              NaN              NaN              NaN   
...                 ...              ...              ...              ...   
335611              NaN              NaN              NaN              NaN   
335612              NaN              NaN              NaN              NaN   
335613              NaN              NaN              NaN              NaN   
335614              NaN              NaN              NaN              NaN   
335615              NaN              NaN              NaN              NaN   

        reactivity_0005  reactivity_0006  reactivity_0007  reac

In [4]:
#for index,row in data.iterrows():
 #   data.loc[index,:].fillna(row.mean())
#assert not data[data[:,4:-1]].isnull().values.any(), "There are still NaNs in the DataFrame!"

In [26]:
bs = pd.DataFrame({'num_legs': [np.nan, 2,np.nan], 'num_wings': [0,np.nan, 2]},
                  )
print(bs)
i = bs.mean(axis=1)
bs.fillna(bs.iloc(0)[0].mean())
print(bs.iloc(0)[0])

   num_legs  num_wings
0       NaN        0.0
1       2.0        NaN
2       NaN        2.0
num_legs     NaN
num_wings    0.0
Name: 0, dtype: float64


In [21]:


reactivity_columns = [col for col in data.columns if col.startswith('reactivity_')]

# Calculate the mean of reactivity values for each sequence
sequence_means = data.groupby('sequence_id')[reactivity_columns].mean()

# Iterate through the reactivity columns and impute missing values with the corresponding sequence mean
for col in reactivity_columns:
    data[col] = data.groupby('sequence_id')[col].transform(lambda x: x.fillna(x.mean()))

# If you want to ensure that no NaNs are left in the DataFrame, you can assert this as follows:
assert not data[reactivity_columns].isnull().values.any(), "There are still NaNs in the DataFrame!"

KeyboardInterrupt: 

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(sequences, reactivity, test_size=0.2, random_state=42)

def encode_sequence(sequence):
    encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'U': [0, 0, 0, 1]}
    encoded_sequence = []
    for nucleotide in sequence:
        # Check if the nucleotide is in the encoding dictionary
        if nucleotide in encoding:
            encoded_sequence.append(encoding[nucleotide])
        else:
            # Handle unexpected characters (e.g., print a warning)
            print(f"Warning: Unexpected character '{nucleotide}' in sequence.")
            # You can choose to replace the unexpected character with a default encoding or ignore it

    # Convert the list of encoded nucleotides to a PyTorch tensor
    return torch.FloatTensor(encoded_sequence)

from torch.nn.utils.rnn import pad_sequence

# Encode sequences
X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad sequences to make them equal length
X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))
X_valid_padded = torch.tensor(pad_sequence(X_valid_encoded, batch_first=True))

C:\Users\matth\AppData\Local\Temp\ipykernel_8796\1744887545.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))
C:\Users\matth\AppData\Local\Temp\ipykernel_8796\1744887545.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_valid_padded = torch.tensor(pad_sequence(X_valid_encoded, batch_first=True))


In [15]:
X_train_padded

tensor([[[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        ...,

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         ...,
 

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output